# API(AEMET)

In [585]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import requests
import json
import os
import numpy as np
import locale
%run functions.ipynb #conecto con mi otro JN y utilizo mis funciones

In [9]:
aemet = obtener_encabezado_api(api_file_path='../api.txt')

La API de la Agencia Estatal de Meteorología (AEMET) es esencial para enriquecer la tabla que ya tengo tratada del INE debido a su capacidad para proporcionar datos climáticos precisos y actualizados. Estos datos permiten comprender cómo las condiciones climáticas influyen en los patrones de turismo y pernoctaciones, y son fundamentales para la planificación, toma de decisiones y análisis más profundos en el sector turístico y otros campos relacionados.

Limpio los datos con las funciones que he creado:

## ENERO22

In [376]:
url_en22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-01-01T00%3A00%3A00UTC/fechafin/2022-01-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

enero22 = solicitud_get(url_en22, api_file_path)
print(enero22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/b338cd98', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [377]:
url_datos_en22= 'https://opendata.aemet.es/opendata/sh/b338cd98'
enero_22= procesar_datos_mes(url_datos_en22, columnas_interes)

In [378]:
enero_22= procesar_y_setear_mes(enero_22, año=2022, mes='enero')
enero_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,10.129289,1.503376,5.338356,6.190377,14.065272,2022,enero
1,ALBACETE,6.384677,0.045968,7.015385,0.026613,12.737097,2022,enero
2,ALICANTE,9.916129,0.433636,6.690323,3.551613,16.283065,2022,enero
3,ALMERIA,11.836585,0.116129,6.777419,7.021138,16.647967,2022,enero
4,ARABA/ALAVA,4.653226,2.538333,5.422581,-1.283871,10.593548,2022,enero


## FEBRERO22

In [372]:
url_feb22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-02-01T00%3A00%3A00UTC/fechafin/2022-02-28T00%3A00%3A00UTC/todasestaciones'

febrero22 = solicitud_get(url_feb22, api_file_path)
print(febrero22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/f7f48dd9', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [373]:
url_datos_feb22=  'https://opendata.aemet.es/opendata/sh/f7f48dd9'
febrero_22= procesar_datos_mes(url_datos_feb22, columnas_interes)

In [375]:
febrero_22 = procesar_y_setear_mes(febrero_22, año=2022, mes='febrero')
febrero_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,10.768304,1.522535,5.409023,6.922321,14.616518,2022,febrero
1,ALBACETE,9.515179,0.190909,7.159615,3.148214,15.877679,2022,febrero
2,ALICANTE,12.166964,0.072222,7.178571,5.994643,18.336607,2022,febrero
3,ALMERIA,13.342727,0.166429,7.192857,8.209091,18.477273,2022,febrero
4,ARABA/ALAVA,6.512500,0.366000,5.073214,-0.242857,13.269643,2022,febrero


## MARZO22

In [369]:
url_mzo22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-03-01T00%3A00%3A00UTC/fechafin/2022-03-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

marzo22 = solicitud_get(url_mzo22, api_file_path)
print(marzo22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/bfeac398', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [370]:
url_datos_mzo22=  'https://opendata.aemet.es/opendata/sh/bfeac398'
marzo_22= procesar_datos_mes(url_datos_mzo22, columnas_interes)

In [371]:
marzo_22 = procesar_y_setear_mes(marzo_22, año=2022, mes='marzo')
marzo_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.490283,3.748899,5.258865,7.617004,15.374597,2022,marzo
1,ALBACETE,9.291129,2.700820,2.983929,5.733871,12.842742,2022,marzo
2,ALICANTE,12.988710,7.066942,3.702151,9.316935,16.670968,2022,marzo
3,ALMERIA,14.205405,2.833077,4.927778,10.381081,18.029054,2022,marzo
4,ARABA/ALAVA,8.561290,1.629508,3.272581,3.990323,13.125806,2022,marzo


## ABRIL22

In [379]:
url_abr22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-04-01T00%3A00%3A00UTC/fechafin/2022-04-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

abril22 = solicitud_get(url_abr22, api_file_path)
print(abril22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/b4d5ad32', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [380]:
url_datos_abr22=  'https://opendata.aemet.es/opendata/sh/b4d5ad32'
abril_22= procesar_datos_mes(url_datos_abr22, columnas_interes)

In [381]:
abril_22 = procesar_y_setear_mes(abril_22, año=2022, mes='abril')
abril_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,12.187611,2.299561,6.859701,8.303097,16.064159,2022,abril
1,ALBACETE,10.927500,3.810345,6.924074,5.479167,16.370833,2022,abril
2,ALICANTE,14.869167,2.708475,7.886667,9.794167,19.945833,2022,abril
3,ALMERIA,15.366443,1.734014,8.213333,10.883221,19.847651,2022,abril
4,ARABA/ALAVA,9.601667,2.714815,5.845000,3.841667,15.368333,2022,abril


## MAYO22

In [382]:
url_my22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-05-01T00%3A00%3A00UTC/fechafin/2022-05-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

mayo22 = solicitud_get(url_my22, api_file_path)
print(mayo22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/8611e676', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [383]:
url_datos_my22=  'https://opendata.aemet.es/opendata/sh/8611e676'
mayo_22= procesar_datos_mes(url_datos_my22, columnas_interes)

In [384]:
mayo_22 = procesar_y_setear_mes(mayo_22, año=2022, mes='mayo')
mayo_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,16.305809,1.342678,7.578472,12.051867,20.556432,2022,mayo
1,ALBACETE,18.918548,0.447967,11.362500,11.329032,26.507258,2022,mayo
2,ALICANTE,20.262727,0.919008,11.537975,13.944545,26.575455,2022,mayo
3,ALMERIA,20.394483,1.441135,11.187097,15.505517,25.275172,2022,mayo
4,ARABA/ALAVA,15.954839,0.389831,8.048387,8.641935,23.275806,2022,mayo


## JUNIO22

In [385]:
url_jun22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-06-01T00%3A00%3A00UTC/fechafin/2022-06-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

junio22 = solicitud_get(url_jun22, api_file_path)
print(junio22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/1e04fc76', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [386]:
url_datos_jun22=  'https://opendata.aemet.es/opendata/sh/1e04fc76'
junio_22= procesar_datos_mes(url_datos_jun22, columnas_interes)

In [387]:
junio_22 = procesar_y_setear_mes(junio_22, año=2022, mes='junio')
junio_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,17.140529,2.204545,6.298592,13.666520,20.613656,2022,junio
1,ALBACETE,24.450420,0.089655,11.862500,16.366387,32.522689,2022,junio
2,ALICANTE,24.780000,0.255085,12.430000,18.069167,31.484167,2022,junio
3,ALMERIA,23.676154,0.050394,10.983051,18.652308,28.700000,2022,junio
4,ARABA/ALAVA,19.431667,0.510909,7.826667,12.016667,26.850000,2022,junio


## JULIO22

In [388]:
url_jul22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-07-01T00%3A00%3A00UTC/fechafin/2022-07-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

julio22 = solicitud_get(url_jul22, api_file_path)
print(julio22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/5dbc83a2', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [389]:
url_datos_jul22=  'https://opendata.aemet.es/opendata/sh/5dbc83a2'
julio_22= procesar_datos_mes(url_datos_jul22, columnas_interes)

In [390]:
julio_22 = procesar_y_setear_mes(julio_22, año=2022, mes='julio')
julio_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,20.661382,0.122785,9.775510,15.865041,25.458943,2022,julio
1,ALBACETE,27.159677,0.350000,12.430645,18.901613,35.417742,2022,julio
2,ALICANTE,27.262903,0.001653,12.211957,20.855645,33.681452,2022,julio
3,ALMERIA,26.990278,0.013571,9.216129,21.835417,32.145833,2022,julio
4,ARABA/ALAVA,20.906452,0.091667,9.348387,12.354839,29.464516,2022,julio


## AGOSTO22

In [391]:
url_ag22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-08-01T00%3A00%3A00UTC/fechafin/2022-08-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

agosto22 = solicitud_get(url_ag22, api_file_path)
print(agosto22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/00aee56b', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [392]:
url_datos_ag22= 'https://opendata.aemet.es/opendata/sh/00aee56b'
agosto_22= procesar_datos_mes(url_datos_ag22, columnas_interes)

In [393]:
agosto_22 = procesar_y_setear_mes(agosto_22, año=2022, mes='agosto')
agosto_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,20.262083,0.400420,8.037162,15.996667,24.530000,2022,agosto
1,ALBACETE,25.786179,0.931707,11.014286,17.927642,33.652846,2022,agosto
2,ALICANTE,27.542742,0.188710,11.206452,21.107258,33.979839,2022,agosto
3,ALMERIA,27.325175,0.088571,6.398387,21.953147,32.699301,2022,agosto
4,ARABA/ALAVA,22.341935,0.213208,8.469355,14.454839,30.235484,2022,agosto


## SEPTIEMBRE22

In [394]:
url_sep22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-09-01T00%3A00%3A00UTC/fechafin/2022-09-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

septiembre22 = solicitud_get(url_sep22, api_file_path)
print(septiembre22)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/ce82dd04', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [395]:
url_datos_sep22= 'https://opendata.aemet.es/opendata/sh/ce82dd04'
septiembre_22= procesar_datos_mes(url_datos_sep22, columnas_interes)

In [396]:
septiembre_22 = procesar_y_setear_mes(septiembre_22, año=2022, mes='septiembre')
septiembre_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,18.676695,3.230568,6.824648,14.812288,22.547458,2022,septiembre
1,ALBACETE,21.633333,0.981667,8.618182,15.055833,28.206667,2022,septiembre
2,ALICANTE,24.683333,2.377119,8.538889,18.810833,30.550000,2022,septiembre
3,ALMERIA,23.904225,0.090345,6.974419,19.039437,28.771127,2022,septiembre
4,ARABA/ALAVA,17.236667,0.650909,6.813333,9.420000,25.053333,2022,septiembre


## OCTUBRE22

In [397]:
url_oct22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-10-01T00%3A00%3A00UTC/fechafin/2022-10-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

octubre22 = solicitud_get(url_oct22, api_file_path)
print(octubre22) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/10e075b0', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [398]:
url_datos_oct22= 'https://opendata.aemet.es/opendata/sh/10e075b0'
octubre_22 = procesar_datos_mes(url_datos_oct22, columnas_interes)

In [399]:
octubre_22 = procesar_y_setear_mes(octubre_22, año=2022, mes='octubre')
octubre_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,17.074797,7.699574,4.360390,13.951220,20.199187,2022,octubre
1,ALBACETE,18.606504,0.519672,5.747170,12.440650,24.782927,2022,octubre
2,ALICANTE,20.960870,0.838983,7.208046,15.174783,26.753913,2022,octubre
3,ALMERIA,21.492523,0.054630,7.119355,16.188785,26.795327,2022,octubre
4,ARABA/ALAVA,17.472581,1.156000,5.466129,10.345161,24.595161,2022,octubre


## NOVIEMBRE22

In [400]:
url_nov22 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-11-01T00%3A00%3A00UTC/fechafin/2022-11-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

noviembre22 = solicitud_get(url_nov22, api_file_path)
print(noviembre22) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/1aaa0a38', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [401]:
url_datos_nov22= 'https://opendata.aemet.es/opendata/sh/1aaa0a38'
noviembre_22= procesar_datos_mes(url_datos_nov22, columnas_interes)

In [402]:
noviembre_22 = procesar_y_setear_mes(noviembre_22, año=2022, mes='noviembre')
noviembre_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,13.310417,6.299571,3.182000,10.267083,16.355417,2022,noviembre
1,ALBACETE,12.128814,0.794737,5.203704,6.997458,17.261017,2022,noviembre
2,ALICANTE,16.556410,1.467227,6.667816,11.092308,22.012821,2022,noviembre
3,ALMERIA,16.593277,0.167164,6.873333,11.676471,21.517647,2022,noviembre
4,ARABA/ALAVA,9.786667,2.984000,3.458333,4.308333,15.261667,2022,noviembre


## DICIEMBRE22

In [403]:
url_dic22= 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-12-01T00%3A00%3A00UTC/fechafin/2022-12-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

diciembre22 = solicitud_get(url_dic22, api_file_path)
print(diciembre22) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/1dbadfee', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [404]:
url_datos_dic22= 'https://opendata.aemet.es/opendata/sh/1dbadfee'
diciembre_22= procesar_datos_mes(url_datos_dic22, columnas_interes)

In [405]:
diciembre_22 = procesar_y_setear_mes(diciembre_22, año=2022, mes='diciembre')
diciembre_22.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.919421,10.436797,2.552903,9.270661,14.563636,2022,diciembre
1,ALBACETE,10.366393,2.361017,3.254545,5.925410,14.795902,2022,diciembre
2,ALICANTE,13.939516,0.532000,4.976344,8.792742,19.090323,2022,diciembre
3,ALMERIA,14.474016,1.621739,5.477419,9.506299,19.447244,2022,diciembre
4,ARABA/ALAVA,8.711290,1.028070,2.946774,4.993548,12.425806,2022,diciembre


## ENERO21

In [406]:
url_e21 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-01-01T00%3A00%3A00UTC/fechafin/2021-01-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

enero21 = solicitud_get(url_e21, api_file_path)
print(enero21)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/986e8bf6', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [407]:
columnas_interes = ['fecha', 'provincia', 'tmed', 'prec', 'sol', 'tmin', 'tmax']
url_datos_enero21=  'https://opendata.aemet.es/opendata/sh/986e8bf6'
enero_21= procesar_datos_mes(url_datos_enero21, columnas_interes)

In [408]:
enero_21 = procesar_y_setear_mes(enero_21, año=2021, mes='enero')
enero_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,8.940244,5.844215,3.199329,6.200000,11.676016,2021,enero
1,ALBACETE,4.436290,1.712281,4.996296,0.243548,8.635484,2021,enero
2,ALICANTE,10.770968,2.059836,6.378495,5.426613,16.119355,2021,enero
3,ALMERIA,11.317143,1.746667,6.545161,6.750714,15.882143,2021,enero
4,ARABA/ALAVA,4.748387,2.807843,2.667742,0.995161,8.511290,2021,enero


## FEBRERO21

In [409]:
url_f21 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-02-01T00%3A00%3A00UTC/fechafin/2021-02-28T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'
febrero21 = solicitud_get(url_f21, api_file_path)
print(febrero21)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/b98e6079', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [410]:
url_datos_febrero21=  'https://opendata.aemet.es/opendata/sh/b98e6079'
febrero_21= procesar_datos_mes(url_datos_febrero21, columnas_interes)

In [411]:
febrero_21 = procesar_y_setear_mes(febrero_21, año=2021, mes='febrero')
febrero_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.546818,8.797209,3.903731,8.729545,14.364545,2021,febrero
1,ALBACETE,9.745536,0.422523,5.934615,4.592857,14.897321,2021,febrero
2,ALICANTE,13.300909,0.095413,6.919048,7.901818,18.702727,2021,febrero
3,ALMERIA,14.025564,0.146392,4.172549,9.425564,18.627068,2021,febrero
4,ARABA/ALAVA,9.103571,1.692453,3.917857,4.367857,13.832143,2021,febrero


## MARZO21

In [412]:
url_mzo21= 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-03-01T00%3A00%3A00UTC/fechafin/2021-03-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'
mzo21 = solicitud_get(url_mzo21, api_file_path)
print(mzo21)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/0cddc92b', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [413]:
url_datos_mzo21=  'https://opendata.aemet.es/opendata/sh/0cddc92b'
marzo_21= procesar_datos_mes(url_datos_mzo21, columnas_interes)

In [414]:
marzo_21 = procesar_y_setear_mes(marzo_21, año=2021, mes='marzo')
marzo_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.585830,0.652941,6.928571,7.182996,15.988664,2021,marzo
1,ALBACETE,9.632258,0.754472,7.020690,3.666129,15.600806,2021,marzo
2,ALICANTE,12.774000,2.648980,7.262319,7.559000,17.980000,2021,marzo
3,ALMERIA,14.368182,1.101613,3.508065,9.658333,19.070455,2021,marzo
4,ARABA/ALAVA,8.358065,1.026316,5.341935,2.670968,14.032258,2021,marzo


## ABRIL21

In [415]:
url_abr21 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-04-01T00%3A00%3A00UTC/fechafin/2021-04-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

abril21 = solicitud_get(url_abr21, api_file_path)
print(abril21)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/5c9e1db9', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [416]:
url_datos_abr21=  'https://opendata.aemet.es/opendata/sh/5c9e1db9'
abril_21= procesar_datos_mes(url_datos_abr21, columnas_interes)

In [417]:
abril_21 = procesar_y_setear_mes(abril_21, año=2021, mes='abril')
abril_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,13.098319,2.184279,6.977119,8.736555,17.453361,2021,abril
1,ALBACETE,12.074167,2.244828,6.937500,6.670833,17.480833,2021,abril
2,ALICANTE,15.032222,2.706098,7.073333,10.364444,19.688889,2021,abril
3,ALMERIA,16.279389,1.177119,5.756410,12.127481,20.426718,2021,abril
4,ARABA/ALAVA,9.511667,1.763158,5.676667,3.533333,15.498333,2021,abril


## MAYO21

In [418]:
url_my21 ='https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-05-01T00%3A00%3A00UTC/fechafin/2021-05-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

my21 = solicitud_get(url_my21, api_file_path)
print(my21)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/3e453900', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [419]:
url_datos_my21=  'https://opendata.aemet.es/opendata/sh/3e453900'
mayo_21= procesar_datos_mes(url_datos_my21, columnas_interes)

In [420]:
mayo_21 = procesar_y_setear_mes(mayo_21, año=2021, mes='mayo')
mayo_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,13.899592,3.461181,8.184615,9.862857,17.933061,2021,mayo
1,ALBACETE,17.479832,1.018261,10.046296,10.863866,24.094118,2021,mayo
2,ALICANTE,19.213445,2.142478,10.540909,13.585714,24.842017,2021,mayo
3,ALMERIA,19.430065,1.170968,5.503571,14.524837,24.342484,2021,mayo
4,ARABA/ALAVA,12.800000,2.774576,7.198387,5.567742,20.043548,2021,mayo


## JUNIO21

In [421]:
url_jn21='https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-06-01T00%3A00%3A00UTC/fechafin/2021-06-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

junio21 = solicitud_get(url_jn21, api_file_path)
print(junio21)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/e4256069', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [422]:
url_datos_jun21=  'https://opendata.aemet.es/opendata/sh/e4256069'
junio_21= procesar_datos_mes(url_datos_jun21, columnas_interes)

In [423]:
junio_21 = procesar_y_setear_mes(junio_21, año=2021, mes='junio')
junio_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,16.472152,2.916810,7.811111,12.662447,20.283122,2021,junio
1,ALBACETE,21.565138,1.614423,10.453571,14.648624,28.484404,2021,junio
2,ALICANTE,22.762500,0.496667,10.828889,17.431667,28.090833,2021,junio
3,ALMERIA,22.950000,0.693220,6.604000,18.141667,27.762879,2021,junio
4,ARABA/ALAVA,17.006667,2.680702,7.040000,10.678333,23.335000,2021,junio


## JULIO21


In [424]:
url_jl21='https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-07-01T00%3A00%3A00UTC/fechafin/2021-07-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

julio21 = solicitud_get(url_jl21, api_file_path)
print(julio21)    

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/b733cc85', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [425]:
url_datos_jul21=  'https://opendata.aemet.es/opendata/sh/b733cc85'
julio_21= procesar_datos_mes(url_datos_jul21, columnas_interes)

In [426]:
julio_21 = procesar_y_setear_mes(julio_21, año=2021, mes='julio')
julio_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,18.231855,0.965106,6.723973,14.704839,21.758871,2021,julio
1,ALBACETE,25.816216,0.004505,12.487097,17.725225,33.909009,2021,julio
2,ALICANTE,25.779032,0.407377,12.139785,19.683065,31.876613,2021,julio
3,ALMERIA,26.011475,0.184677,8.088889,20.853279,31.165574,2021,julio
4,ARABA/ALAVA,18.795161,0.066667,7.887097,11.854839,25.737097,2021,julio


## AGOSTO21

In [427]:
url_ag21='https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-08-01T00%3A00%3A00UTC/fechafin/2021-08-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

agosto21 = solicitud_get(url_ag21, api_file_path)
print(agosto21)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/e491ac07', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [428]:
url_datos_ag21= 'https://opendata.aemet.es/opendata/sh/e491ac07'
agosto_21= procesar_datos_mes(url_datos_ag21, columnas_interes)

In [429]:
agosto_21 = procesar_y_setear_mes(agosto_21, año=2021, mes='agosto')
agosto_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,19.207692,0.541026,8.318792,15.094737,23.321053,2021,agosto
1,ALBACETE,26.107258,1.037190,10.096226,18.595968,33.614516,2021,agosto
2,ALICANTE,26.900806,0.013223,9.769892,21.286290,32.508871,2021,agosto
3,ALMERIA,27.076613,0.046721,9.690323,22.304032,31.847581,2021,agosto
4,ARABA/ALAVA,19.269355,0.347541,7.912903,12.012903,26.525806,2021,agosto


## SEPTIEMBRE21

In [430]:
url_sep21 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-09-01T00%3A00%3A00UTC/fechafin/2021-09-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

septiembre21 = solicitud_get(url_sep21, api_file_path)
print(septiembre21) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/16508fca', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [431]:
url_datos_sep21= 'https://opendata.aemet.es/opendata/sh/16508fca'
septiembre_21 = procesar_datos_mes(url_datos_sep21, columnas_interes)

In [432]:
septiembre_21 = procesar_y_setear_mes(septiembre_21, año=2021, mes='septiembre')
septiembre_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,18.563404,2.992411,6.838732,14.752340,22.368511,2021,septiembre
1,ALBACETE,21.185000,1.835897,8.240000,15.124167,27.255000,2021,septiembre
2,ALICANTE,24.458824,1.309483,8.576667,19.005042,29.904202,2021,septiembre
3,ALMERIA,24.738889,0.068908,9.103333,20.005556,29.463889,2021,septiembre
4,ARABA/ALAVA,18.433333,1.828571,5.290000,12.130000,24.728333,2021,septiembre


## OCTUBRE21

In [433]:
url_oct21 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-10-01T00%3A00%3A00UTC/fechafin/2021-10-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

octubre21 = solicitud_get(url_oct21, api_file_path)
print(octubre21) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/7a619e93', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [434]:
url_datos_oct21= 'https://opendata.aemet.es/opendata/sh/7a619e93'
octubre_21 = procesar_datos_mes(url_datos_oct21, columnas_interes)

In [435]:
octubre_21 = procesar_y_setear_mes(octubre_21, año=2021, mes='octubre')
octubre_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,15.848178,4.034348,6.020690,11.961943,19.734413,2021,octubre
1,ALBACETE,16.633065,1.369106,7.454545,10.691935,22.568548,2021,octubre
2,ALICANTE,19.454032,1.847934,7.444086,14.068548,24.837903,2021,octubre
3,ALMERIA,19.749438,0.264167,8.174194,14.639326,24.869663,2021,octubre
4,ARABA/ALAVA,12.217742,0.960714,5.782258,4.719355,19.714516,2021,octubre


## NOVIEMBRE21

In [436]:
url_nov21 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-11-01T00%3A00%3A00UTC/fechafin/2021-11-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

noviembre21 = solicitud_get(url_nov21, api_file_path)
print(noviembre21) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/9fca8e37', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [437]:
url_datos_nov21= 'https://opendata.aemet.es/opendata/sh/9fca8e37'
noviembre_21 = procesar_datos_mes(url_datos_nov21, columnas_interes)

In [438]:
noviembre_21 = procesar_y_setear_mes(noviembre_21, año=2021, mes='noviembre')
noviembre_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.236820,2.289823,5.308088,7.805021,14.674059,2021,noviembre
1,ALBACETE,8.404202,0.983051,6.032727,3.588235,13.224370,2021,noviembre
2,ALICANTE,13.713559,3.088793,5.867778,8.903390,18.516949,2021,noviembre
3,ALMERIA,13.871429,0.207874,7.840000,9.285714,18.452747,2021,noviembre
4,ARABA/ALAVA,7.193333,5.978947,2.370000,3.793333,10.591667,2021,noviembre


## DICIEMBRE21

In [439]:
url_dic21= 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2021-12-01T00%3A00%3A00UTC/fechafin/2021-12-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

diciembre21 = solicitud_get(url_dic21, api_file_path)
print(diciembre21) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/18089b18', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [440]:
url_datos_dic21= 'https://opendata.aemet.es/opendata/sh/18089b18'
diciembre_21= procesar_datos_mes(url_datos_dic21, columnas_interes)

In [441]:
diciembre_21 = procesar_y_setear_mes(diciembre_21, año=2021, mes='diciembre')
diciembre_21.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,12.082857,4.536929,3.425517,9.235510,14.923673,2021,diciembre
1,ALBACETE,8.972581,0.352066,5.281132,4.372581,13.582258,2021,diciembre
2,ALICANTE,13.292742,0.051240,5.622581,8.221774,18.371774,2021,diciembre
3,ALMERIA,13.890323,0.069655,5.983871,9.392742,18.392742,2021,diciembre
4,ARABA/ALAVA,6.579032,6.454386,3.069355,2.900000,10.259677,2021,diciembre


## ENERO20

In [442]:
url_en20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-01-01T00%3A00%3A00UTC/fechafin/2020-01-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

enero20 = solicitud_get(url_en20, api_file_path)
print(enero20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/86664742', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [443]:
url_datos_en20= 'https://opendata.aemet.es/opendata/sh/86664742'
enero_20= procesar_datos_mes(url_datos_en20, columnas_interes)

In [444]:
enero_20 = procesar_y_setear_mes(enero_20, año=2020, mes='enero')
enero_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,10.294118,6.768947,3.769828,7.373039,13.210294,2020,enero
1,ALBACETE,6.709917,1.557851,5.264815,1.499174,11.919835,2020,enero
2,ALICANTE,10.698374,2.205042,6.613953,4.880488,16.519512,2020,enero
3,ALMERIA,11.479861,0.850667,6.409677,6.564583,16.393056,2020,enero
4,ARABA/ALAVA,5.338710,1.067857,3.430645,0.751613,9.929032,2020,enero


## FEBRERO20

In [445]:
url_feb20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-02-01T00%3A00%3A00UTC/fechafin/2020-02-29T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

febrero20 = solicitud_get(url_feb20, api_file_path)
print(febrero20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/56d7bb3d', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [446]:
url_datos_feb20=  'https://opendata.aemet.es/opendata/sh/56d7bb3d'
febrero_20= procesar_datos_mes(url_datos_feb20, columnas_interes)

In [447]:
febrero_20 = procesar_y_setear_mes(febrero_20, año=2020, mes='febrero')
febrero_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,12.005607,2.291429,4.735398,8.682710,15.328972,2020,febrero
1,ALBACETE,11.039474,0.026316,7.000000,4.828947,17.241228,2020,febrero
2,ALICANTE,13.914655,0.065179,8.223256,7.398276,20.432759,2020,febrero
3,ALMERIA,14.179167,0.011888,7.886207,8.941667,19.415972,2020,febrero
4,ARABA/ALAVA,8.713793,0.340000,5.687931,2.082759,15.355172,2020,febrero


## MARZO20

In [448]:
url_mzo20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-03-01T00%3A00%3A00UTC/fechafin/2020-03-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

marzo20 = solicitud_get(url_mzo20, api_file_path)
print(marzo20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/32dbe22f', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [449]:
url_datos_mzo20=  'https://opendata.aemet.es/opendata/sh/32dbe22f'
marzo_20= procesar_datos_mes(url_datos_mzo20, columnas_interes)

In [450]:
marzo_20 = procesar_y_setear_mes(marzo_20, año=2020, mes='marzo')
marzo_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.459350,4.151376,5.730986,8.051626,14.868699,2020,marzo
1,ALBACETE,10.226271,4.284615,5.523729,5.063559,15.387288,2020,marzo
2,ALICANTE,14.154032,3.393333,6.230108,9.204032,19.095161,2020,marzo
3,ALMERIA,14.942384,0.804575,6.460000,10.615894,19.261589,2020,marzo
4,ARABA/ALAVA,8.316129,2.576667,5.687097,1.974194,14.659677,2020,marzo


## ABRIL20

In [451]:
url_abr20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-04-01T00%3A00%3A00UTC/fechafin/2020-04-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

abril20 = solicitud_get(url_abr20, api_file_path)
print(abril20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/4b1fc5d3', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [452]:
url_datos_abr20=  'https://opendata.aemet.es/opendata/sh/4b1fc5d3'
abril_20= procesar_datos_mes(url_datos_abr20, columnas_interes)

In [453]:
abril_20 = procesar_y_setear_mes(abril_20, año=2020, mes='abril')
abril_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,13.406356,4.371359,3.888667,10.103814,16.700424,2020,abril
1,ALBACETE,13.052137,1.822523,6.310000,7.941026,18.165812,2020,abril
2,ALICANTE,15.711017,1.649558,7.545556,10.855085,20.568644,2020,abril
3,ALMERIA,16.114286,0.737584,NaN,12.145578,20.080952,2020,abril
4,ARABA/ALAVA,12.736667,2.325000,5.290000,7.346667,18.121667,2020,abril


## MAYO20

In [454]:
url_my20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-05-01T00%3A00%3A00UTC/fechafin/2020-05-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

mayo20 = solicitud_get(url_my20, api_file_path)
print(mayo20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/f0ccfc72', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [455]:
url_datos_my20=  'https://opendata.aemet.es/opendata/sh/f0ccfc72'
mayo_20= procesar_datos_mes(url_datos_my20, columnas_interes)

In [456]:
mayo_20 = procesar_y_setear_mes(mayo_20, año=2020, mes='mayo')
mayo_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,16.840081,1.074678,8.495364,12.589069,21.094332,2020,mayo
1,ALBACETE,17.870968,1.876423,9.638710,11.278226,24.459677,2020,mayo
2,ALICANTE,20.310484,1.005785,10.875269,14.337097,26.284677,2020,mayo
3,ALMERIA,20.653247,0.154839,11.444444,15.663636,25.646104,2020,mayo
4,ARABA/ALAVA,15.438710,3.529825,8.222581,8.043548,22.838710,2020,mayo


## JUNIO20

In [457]:
url_jun20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-06-01T00%3A00%3A00UTC/fechafin/2020-06-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

junio20 = solicitud_get(url_jun20, api_file_path)
print(junio20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/6de2a2e1', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [458]:
url_datos_jun20=  'https://opendata.aemet.es/opendata/sh/6de2a2e1'
junio_20= procesar_datos_mes(url_datos_jun20, columnas_interes)

In [459]:
junio_20 = procesar_y_setear_mes(junio_20, año=2021, mes='junio')
junio_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,16.626360,1.387387,6.479021,13.132636,20.126778,2021,junio
1,ALBACETE,22.353774,0.099029,11.423214,14.876415,29.838679,2021,junio
2,ALICANTE,22.915385,0.379091,11.830682,16.911966,28.925641,2021,junio
3,ALMERIA,22.316892,0.076522,12.610000,17.477027,27.152703,2021,junio
4,ARABA/ALAVA,16.830000,1.800000,6.895000,10.446667,23.205000,2021,junio


## JULIO20

In [460]:
url_jul20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-07-01T00%3A00%3A00UTC/fechafin/2020-07-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

julio20 = solicitud_get(url_jul20, api_file_path)
print(julio20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/46e0f9b8', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [461]:
url_datos_jul20= 'https://opendata.aemet.es/opendata/sh/46e0f9b8'
julio_20= procesar_datos_mes(url_datos_jul20, columnas_interes)

In [462]:
julio_20 = procesar_y_setear_mes(julio_20, año=2020, mes='julio')
julio_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,19.745968,0.051915,10.102027,15.214516,24.279839,2020,julio
1,ALBACETE,26.603000,0.543299,11.809677,18.404000,34.806000,2020,julio
2,ALICANTE,26.208871,0.038655,12.067742,20.375000,32.050000,2020,julio
3,ALMERIA,26.865753,0.006087,12.083871,22.039726,31.693151,2020,julio
4,ARABA/ALAVA,19.537097,0.315254,9.232258,11.974194,27.093548,2020,julio


## AGOSTO20

In [463]:
url_ag20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-08-01T00%3A00%3A00UTC/fechafin/2020-08-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

agosto20 = solicitud_get(url_ag20, api_file_path)
print(agosto20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/c9158638', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [464]:
url_datos_ag20= 'https://opendata.aemet.es/opendata/sh/c9158638'
agosto_20= procesar_datos_mes(url_datos_ag20, columnas_interes)

In [465]:
agosto_20 = procesar_y_setear_mes(agosto_20, año=2020, mes='agosto')
agosto_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,19.467073,3.038912,7.091558,15.711789,23.221545,2020,agosto
1,ALBACETE,26.240426,0.200000,11.183636,18.297872,34.198936,2020,agosto
2,ALICANTE,26.675806,0.381513,10.959140,20.513710,32.848387,2020,agosto
3,ALMERIA,26.666000,0.015054,11.261290,21.705333,31.620667,2020,agosto
4,ARABA/ALAVA,20.204839,1.431481,7.590323,12.262903,28.150000,2020,agosto


## SEPTIEMBRE20

In [466]:
url_sep20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-09-01T00%3A00%3A00UTC/fechafin/2020-09-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

septiembre20 = solicitud_get(url_sep20, api_file_path)
print(septiembre20)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/c8dec084', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [467]:
url_datos_sep20= 'https://opendata.aemet.es/opendata/sh/c8dec084'
septiembre_20= procesar_datos_mes(url_datos_sep20, columnas_interes)

In [468]:
septiembre_20 = procesar_y_setear_mes(septiembre_20, año=2020, mes='septiembre')
septiembre_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,18.763333,1.307339,7.921918,14.425833,23.101250,2020,septiembre
1,ALBACETE,20.784091,0.748864,9.035185,13.918182,27.642045,2020,septiembre
2,ALICANTE,22.946667,0.165812,9.438202,17.082500,28.813333,2020,septiembre
3,ALMERIA,23.445270,0.263333,9.013333,18.743243,28.154054,2020,septiembre
4,ARABA/ALAVA,17.200000,2.424528,7.303333,9.516667,24.900000,2020,septiembre


## OCTUBRE20

In [469]:
url_oct20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-10-01T00%3A00%3A00UTC/fechafin/2020-10-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

octubre20 = solicitud_get(url_oct20, api_file_path)
print(octubre20) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/23c6bfc9', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [470]:
url_datos_oct20= 'https://opendata.aemet.es/opendata/sh/23c6bfc9'
octubre_20 = procesar_datos_mes(url_datos_oct20, columnas_interes)

In [471]:
octubre_20 = procesar_y_setear_mes(octubre_20, año=2020, mes='octubre')
octubre_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,14.561633,4.365417,4.166216,11.576735,17.548163,2020,octubre
1,ALBACETE,13.984553,0.863115,8.110526,7.540650,20.418699,2020,octubre
2,ALICANTE,18.218548,0.225203,8.765591,11.660484,24.777419,2020,octubre
3,ALMERIA,18.167333,0.111828,8.664516,12.884667,23.444000,2020,octubre
4,ARABA/ALAVA,11.862903,3.833898,4.341935,6.556452,17.177419,2020,octubre


## NOVIEMBRE20

In [472]:
url_nov20 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-11-01T00%3A00%3A00UTC/fechafin/2020-11-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

noviembre20 = solicitud_get(url_nov20, api_file_path)
print(noviembre20) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/543d2a91', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [473]:
url_datos_nov20= 'https://opendata.aemet.es/opendata/sh/543d2a91'
noviembre_20 = procesar_datos_mes(url_datos_nov20, columnas_interes)

In [474]:
noviembre_20 = procesar_y_setear_mes(noviembre_20, año=2020, mes='noviembre')
noviembre_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,13.431381,4.028511,4.401361,10.237238,16.632636,2020,noviembre
1,ALBACETE,11.836667,2.661667,5.826415,6.506667,17.160833,2020,noviembre
2,ALICANTE,15.717544,0.998165,6.610465,10.228070,21.203509,2020,noviembre
3,ALMERIA,16.323448,1.281818,6.930000,11.704828,20.940690,2020,noviembre
4,ARABA/ALAVA,9.746667,1.058929,4.533333,4.360000,15.130000,2020,noviembre


## DICIEMBRE20

In [475]:
url_dic20= 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2020-12-01T00%3A00%3A00UTC/fechafin/2020-12-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

diciembre20 = solicitud_get(url_dic20, api_file_path)
print(diciembre20) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/550f538f', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [476]:
url_datos_dic20= 'https://opendata.aemet.es/opendata/sh/550f538f'
diciembre_20= procesar_datos_mes(url_datos_dic20, columnas_interes)

In [477]:
diciembre_20 = procesar_y_setear_mes(diciembre_20, año=2020, mes='diciembre')
diciembre_20.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,10.233878,7.775207,2.098649,7.640816,12.828571,2020,diciembre
1,ALBACETE,6.778226,0.714634,5.110000,2.020968,11.525000,2020,diciembre
2,ALICANTE,11.990244,0.189167,6.617204,6.378049,17.597561,2020,diciembre
3,ALMERIA,12.791429,0.037634,7.093548,8.021429,17.557857,2020,diciembre
4,ARABA/ALAVA,6.256452,5.535088,2.387097,2.927419,9.590323,2020,diciembre


## ENERO19

In [478]:
url_en19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-01-01T00%3A00%3A00UTC/fechafin/2019-01-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

enero19 = solicitud_get(url_en19, api_file_path)
print(enero19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/4921b452', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [479]:
url_datos_en19= 'https://opendata.aemet.es/opendata/sh/4921b452'
enero_19= procesar_datos_mes(url_datos_en19, columnas_interes)

In [480]:
enero_19 = procesar_y_setear_mes(enero_19, año=2019, mes='enero')
enero_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,9.088710,3.441304,3.798529,5.790726,12.392339,2019,enero
1,ALBACETE,6.612903,0.131818,6.670909,0.508602,12.716129,2019,enero
2,ALICANTE,10.133871,0.091736,6.831183,4.155645,16.129839,2019,enero
3,ALMERIA,12.023810,0.050336,7.603226,6.736054,17.312925,2019,enero
4,ARABA/ALAVA,4.633871,6.223729,2.580645,0.650000,8.616129,2019,enero


## FEBRERO19

In [481]:
url_feb19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-02-01T00%3A00%3A00UTC/fechafin/2019-02-28T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

febrero19 = solicitud_get(url_feb19, api_file_path)
print(febrero19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/188ecd8a', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [482]:
url_datos_feb19=  'https://opendata.aemet.es/opendata/sh/188ecd8a'
febrero_19= procesar_datos_mes(url_datos_feb19, columnas_interes)

In [483]:
febrero_19 = procesar_y_setear_mes(febrero_19, año=2019, mes='febrero')
febrero_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.364414,1.770673,6.068254,7.122072,15.616667,2019,febrero
1,ALBACETE,9.293548,0.116304,8.723077,2.065591,16.519355,2019,febrero
2,ALICANTE,10.861607,0.055357,9.092857,4.135714,17.591964,2019,febrero
3,ALMERIA,12.138655,0.055714,8.807143,6.531933,17.739496,2019,febrero
4,ARABA/ALAVA,7.067857,1.883333,6.782143,-0.194643,14.319643,2019,febrero


## MARZO19

In [484]:
url_mzo19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-03-01T00%3A00%3A00UTC/fechafin/2019-03-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

marzo19 = solicitud_get(url_mzo19, api_file_path)
print(marzo19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/a23abaa4', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [485]:
url_datos_mzo19=  'https://opendata.aemet.es/opendata/sh/a23abaa4'
marzo_19= procesar_datos_mes(url_datos_mzo19, columnas_interes)

In [486]:
marzo_19 = procesar_y_setear_mes(marzo_19, año=2019, mes='marzo')
marzo_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.691532,2.531381,6.835294,7.400403,15.987500,2019,marzo
1,ALBACETE,11.173387,0.684298,8.885714,3.891935,18.456452,2019,marzo
2,ALICANTE,12.740650,0.828333,8.707527,6.112195,19.363415,2019,marzo
3,ALMERIA,14.581102,0.278431,8.529032,9.638583,19.523622,2019,marzo
4,ARABA/ALAVA,8.542623,0.677193,7.703226,1.308197,15.780328,2019,marzo


## ABRIL19

In [487]:
url_abr19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-04-01T00%3A00%3A00UTC/fechafin/2019-04-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

abril19 = solicitud_get(url_abr19, api_file_path)
print(abril19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/8dc909ad', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [488]:
url_datos_abr19=  'https://opendata.aemet.es/opendata/sh/8dc909ad'
abril_19= procesar_datos_mes(url_datos_abr19, columnas_interes)

In [489]:
abril_19 = procesar_y_setear_mes(abril_19, año=2019, mes='abril')
abril_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,12.008017,4.688106,6.258519,8.148101,15.865823,2019,abril
1,ALBACETE,11.467521,4.981739,7.768519,6.119658,16.817094,2019,abril
2,ALICANTE,14.464957,6.569027,7.941111,9.428205,19.505128,2019,abril
3,ALMERIA,15.465101,1.325676,9.146667,11.003356,19.929530,2019,abril
4,ARABA/ALAVA,9.975000,2.286441,5.526667,3.906667,16.045000,2019,abril


## MAYO19

In [490]:
url_my19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-05-01T00%3A00%3A00UTC/fechafin/2019-05-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

mayo19 = solicitud_get(url_my19, api_file_path)
print(mayo19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/3227ed55', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [491]:
url_datos_my19=  'https://opendata.aemet.es/opendata/sh/3227ed55'
mayo_19= procesar_datos_mes(url_datos_my19, columnas_interes)

In [492]:
mayo_19 = procesar_y_setear_mes(mayo_19, año=2019, mes='mayo')
mayo_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,14.989474,1.383128,8.838235,10.632794,19.340891,2019,mayo
1,ALBACETE,17.135484,0.235772,10.617857,10.009677,24.262903,2019,mayo
2,ALICANTE,18.205691,0.241803,10.668817,12.524390,23.891870,2019,mayo
3,ALMERIA,19.587662,0.047097,11.077419,14.905844,24.272078,2019,mayo
4,ARABA/ALAVA,11.848387,1.646552,7.029032,4.982258,18.714516,2019,mayo


## JUNIO19

In [493]:
url_jun19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-06-01T00%3A00%3A00UTC/fechafin/2019-06-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

junio19 = solicitud_get(url_jun19, api_file_path)
print(junio19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/cae4a15a', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [494]:
url_datos_jun19=  'https://opendata.aemet.es/opendata/sh/cae4a15a'
junio_19= procesar_datos_mes(url_datos_jun19, columnas_interes)

In [495]:
junio_19 = procesar_y_setear_mes(junio_19, año=2019, mes='junio')
junio_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,15.702252,2.460526,6.946923,11.907207,19.496396,2019,junio
1,ALBACETE,22.074167,0.266667,12.235185,13.852500,30.287500,2019,junio
2,ALICANTE,22.269748,0.038136,12.040000,16.087395,28.453782,2019,junio
3,ALMERIA,21.950000,0.042953,12.096667,17.104795,26.795205,2019,junio
4,ARABA/ALAVA,18.113333,0.441071,9.001667,9.591667,26.620000,2019,junio


## JULIO19

In [496]:
url_jul19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-07-01T00%3A00%3A00UTC/fechafin/2019-07-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

julio19 = solicitud_get(url_jul19, api_file_path)
print(julio19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/e1025e62', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [497]:
url_datos_jul19=  'https://opendata.aemet.es/opendata/sh/e1025e62'
julio_19= procesar_datos_mes(url_datos_jul19, columnas_interes)

In [498]:
julio_19 = procesar_y_setear_mes(julio_19, año=2019, mes='julio')
julio_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,19.202128,0.529787,8.688652,15.191489,23.220000,2019,julio
1,ALBACETE,25.976613,0.518852,11.870968,18.188710,33.763710,2019,julio
2,ALICANTE,26.481148,0.040678,11.572043,20.730328,32.228689,2019,julio
3,ALMERIA,25.873203,0.003247,11.564516,21.041558,30.720261,2019,julio
4,ARABA/ALAVA,20.495161,1.675439,8.156452,13.095161,27.896774,2019,julio


## AGOSTO19

In [499]:
url_ag19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-08-01T00%3A00%3A00UTC/fechafin/2019-08-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

agosto19 = solicitud_get(url_ag19, api_file_path)
print(agosto19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/512795bb', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [500]:
url_datos_ag19= 'https://opendata.aemet.es/opendata/sh/512795bb'
agosto_19= procesar_datos_mes(url_datos_ag19, columnas_interes)

In [501]:
agosto_19 = procesar_y_setear_mes(agosto_19, año=2019, mes='agosto')
agosto_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,19.656897,1.407075,7.630645,15.500431,23.815517,2019,agosto
1,ALBACETE,25.584874,0.411017,10.666667,17.718487,33.452941,2019,agosto
2,ALICANTE,26.165546,1.928333,10.908889,20.344538,31.988235,2019,agosto
3,ALMERIA,26.145455,0.014935,10.974194,21.464286,30.829221,2019,agosto
4,ARABA/ALAVA,20.595161,0.923636,8.662903,13.187097,28.000000,2019,agosto


## SEPTIEMBRE19

In [502]:
url_sep19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-09-01T00%3A00%3A00UTC/fechafin/2019-09-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

septiembre19 = solicitud_get(url_sep19, api_file_path)
print(septiembre19)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/cfd9dc76', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [503]:
url_datos_sep19= 'https://opendata.aemet.es/opendata/sh/cfd9dc76'
septiembre_19= procesar_datos_mes(url_datos_sep19, columnas_interes)

In [504]:
septiembre_19 = procesar_y_setear_mes(septiembre_19, año=2019, mes='septiembre')
septiembre_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,18.445673,1.696000,7.462264,14.317788,22.573558,2019,septiembre
1,ALBACETE,20.938462,2.277119,7.974545,15.021368,26.858120,2019,septiembre
2,ALICANTE,23.159167,5.114953,8.364045,18.284167,28.024167,2019,septiembre
3,ALMERIA,23.789189,1.740667,8.523333,19.472297,28.100676,2019,septiembre
4,ARABA/ALAVA,17.141667,1.418966,6.915000,10.250000,24.036667,2019,septiembre


## OCTUBRE19

In [505]:
url_oct19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-10-01T00%3A00%3A00UTC/fechafin/2019-10-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

octubre19 = solicitud_get(url_oct19, api_file_path)
print(octubre19) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/ba3b9a6c', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [506]:
url_datos_oct19= 'https://opendata.aemet.es/opendata/sh/ba3b9a6c'
octubre_19 = procesar_datos_mes(url_datos_oct19, columnas_interes)

In [507]:
octubre_19 = procesar_y_setear_mes(octubre_19, año=2019, mes='octubre')
octubre_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,15.442254,5.911594,4.044643,12.131455,18.741315,2019,octubre
1,ALBACETE,16.732231,1.331034,7.756604,10.695868,22.774380,2019,octubre
2,ALICANTE,19.267480,1.571545,8.441935,13.505691,25.023577,2019,octubre
3,ALMERIA,19.424138,0.735526,8.525806,14.342069,24.506897,2019,octubre
4,ARABA/ALAVA,14.488525,1.750000,5.604839,8.355738,20.632787,2019,octubre


## NOVIEMBRE19

In [508]:
url_nov19 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-11-01T00%3A00%3A00UTC/fechafin/2019-11-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

noviembre19 = solicitud_get(url_nov19, api_file_path)
print(noviembre19) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/b57319d7', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [509]:
url_datos_nov19= 'https://opendata.aemet.es/opendata/sh/b57319d7'
noviembre_19 = procesar_datos_mes(url_datos_nov19, columnas_interes)

In [510]:
noviembre_19 = procesar_y_setear_mes(noviembre_19, año=2019, mes='noviembre')
noviembre_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.531841,9.725126,1.725714,9.056219,14.005970,2019,noviembre
1,ALBACETE,9.603509,1.343243,4.811111,5.712281,13.491228,2019,noviembre
2,ALICANTE,14.652500,0.800840,7.308889,9.755833,19.555000,2019,noviembre
3,ALMERIA,14.647297,0.685294,6.956667,10.315541,18.979730,2019,noviembre
4,ARABA/ALAVA,8.538333,8.127586,2.076667,4.933333,12.121667,2019,noviembre


## DICIEMBRE19

In [511]:
url_dic19= 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2019-12-01T00%3A00%3A00UTC/fechafin/2019-12-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

diciembre19 = solicitud_get(url_dic19, api_file_path)
print(diciembre19) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/f4610c81', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [512]:
url_datos_dic19= 'https://opendata.aemet.es/opendata/sh/f4610c81'
diciembre_19= procesar_datos_mes(url_datos_dic19, columnas_interes)

In [513]:
diciembre_19 = procesar_y_setear_mes(diciembre_19, año=2019, mes='diciembre')
diciembre_19.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,10.770335,7.096078,3.263717,7.733971,13.805263,2019,diciembre
1,ALBACETE,8.819130,1.376991,4.288889,4.589565,13.049565,2019,diciembre
2,ALICANTE,13.031452,2.489076,5.381720,7.960484,18.105645,2019,diciembre
3,ALMERIA,13.610526,0.782517,5.677419,8.779699,18.431579,2019,diciembre
4,ARABA/ALAVA,6.743548,2.942857,2.685484,2.224194,11.269355,2019,diciembre


## ENERO18

In [514]:
url_en18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-01-01T00%3A00%3A00UTC/fechafin/2018-01-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

enero18 = solicitud_get(url_en18, api_file_path)
print(enero18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/4cb844f4', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [515]:
url_datos_en18= 'https://opendata.aemet.es/opendata/sh/4cb844f4'
enero_18= procesar_datos_mes(url_datos_en18, columnas_interes)

In [516]:
enero_18 = procesar_y_setear_mes(enero_18, año=2018, mes='enero')
enero_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,10.068163,3.769456,2.528467,7.360408,12.773469,2018,enero
1,ALBACETE,7.240323,1.161789,5.959649,2.165323,12.312903,2018,enero
2,ALICANTE,11.533871,2.181197,7.197849,6.019355,17.037903,2018,enero
3,ALMERIA,12.171895,0.392763,6.748387,7.255556,17.081699,2018,enero
4,ARABA/ALAVA,6.133871,4.018033,2.643548,2.108065,10.162903,2018,enero


## FEBRERO18

In [517]:
url_feb18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-02-01T00%3A00%3A00UTC/fechafin/2018-02-28T00%3A00%3A00UTC/todasestaciones'

febrero18 = solicitud_get(url_feb18, api_file_path)
print(febrero18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/efe9ffd6', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [518]:
url_datos_feb18=  'https://opendata.aemet.es/opendata/sh/efe9ffd6'
febrero_18= procesar_datos_mes(url_datos_feb18, columnas_interes)

In [519]:
febrero_18 = procesar_y_setear_mes(febrero_18, año=2018, mes='febrero')
febrero_18.head()

TypeError: 'NoneType' object is not subscriptable

## MARZO18

In [522]:
url_mzo18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-03-01T00%3A00%3A00UTC/fechafin/2018-03-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

marzo18 = solicitud_get(url_mzo18, api_file_path)
print(marzo18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/3268a413', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [523]:
url_datos_mzo18=  'https://opendata.aemet.es/opendata/sh/3268a413'
marzo_18= procesar_datos_mes(url_datos_mzo18, columnas_interes)

In [524]:
marzo_18 = procesar_y_setear_mes(marzo_18, año=2018, mes='marzo')
marzo_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,9.337959,7.681780,3.750388,6.058776,12.623673,2018,marzo
1,ALBACETE,8.905645,1.706838,6.288333,4.174194,13.645968,2018,marzo
2,ALICANTE,13.057258,0.808621,7.337778,7.661290,18.453226,2018,marzo
3,ALMERIA,13.696667,1.901600,7.270968,9.570000,17.832500,2018,marzo
4,ARABA/ALAVA,7.946774,2.750847,4.769355,3.441935,12.450000,2018,marzo


## ABRIL18

In [527]:
url_abr18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-04-01T00%3A00%3A00UTC/fechafin/2018-04-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

abril18 = solicitud_get(url_abr18, api_file_path)
print(abril18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/de263bee', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [528]:
url_datos_abr18=  'https://opendata.aemet.es/opendata/sh/de263bee'
abril_18= procesar_datos_mes(url_datos_abr18, columnas_interes)

In [529]:
abril_18 = procesar_y_setear_mes(abril_18, año=2018, mes='abril')
abril_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,12.369231,3.871681,6.020968,8.676923,16.060684,2018,abril
1,ALBACETE,12.586441,1.064912,7.311111,6.808475,18.365254,2018,abril
2,ALICANTE,15.486667,0.811304,8.816667,10.012500,20.960000,2018,abril
3,ALMERIA,15.936036,0.400000,8.530000,11.006306,20.871171,2018,abril
4,ARABA/ALAVA,11.706667,3.929310,5.868333,6.125000,17.310000,2018,abril


## MAYO18

In [530]:
url_my18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-05-01T00%3A00%3A00UTC/fechafin/2018-05-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

mayo18 = solicitud_get(url_my18, api_file_path)
print(mayo18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/c1889823', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [531]:
url_datos_my18=  'https://opendata.aemet.es/opendata/sh/c1889823'
mayo_18= procesar_datos_mes(url_datos_my18, columnas_interes)

In [532]:
mayo_18 = procesar_y_setear_mes(mayo_18, año=2018, mes='mayo')
mayo_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,14.513878,0.953448,7.141221,10.636735,18.380408,2018,mayo
1,ALBACETE,16.001613,1.914407,8.929825,9.309677,22.691129,2018,mayo
2,ALICANTE,17.824194,0.437705,11.055914,12.179839,23.475000,2018,mayo
3,ALMERIA,18.320833,0.147368,10.780645,13.611111,23.022917,2018,mayo
4,ARABA/ALAVA,12.561290,1.905263,5.254839,6.664516,18.462903,2018,mayo


## JUNIO18

In [533]:
url_jun18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-06-01T00%3A00%3A00UTC/fechafin/2018-06-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

junio18 = solicitud_get(url_jun18, api_file_path)
print(junio18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/e5a68a26', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [534]:
url_datos_jun18=  'https://opendata.aemet.es/opendata/sh/e5a68a26'
junio_18= procesar_datos_mes(url_datos_jun18, columnas_interes)

In [535]:
junio_18 = procesar_y_setear_mes(junio_18, año=2018, mes='junio')
junio_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,17.012766,3.060699,6.017323,13.600851,20.420851,2018,junio
1,ALBACETE,20.758333,1.621008,10.818182,13.981667,27.540000,2018,junio
2,ALICANTE,22.212605,1.241026,11.290000,16.554622,27.873109,2018,junio
3,ALMERIA,22.151408,0.124107,11.726667,17.355634,26.939437,2018,junio
4,ARABA/ALAVA,17.380000,1.969811,6.801667,11.325000,23.438333,2018,junio


## JULIO18

In [536]:
url_jul18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-07-01T00%3A00%3A00UTC/fechafin/2018-07-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

julio18 = solicitud_get(url_jul18, api_file_path)
print(julio18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/79909d39', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [537]:
url_datos_jul18=  'https://opendata.aemet.es/opendata/sh/79909d39'
julio_18= procesar_datos_mes(url_datos_jul18, columnas_interes)

In [538]:
julio_18 = procesar_y_setear_mes(julio_18, año=2018, mes='julio')
julio_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,19.324066,1.054894,6.597692,15.859336,22.787552,2018,julio
1,ALBACETE,25.502419,0.064228,13.034483,17.388710,33.624194,2018,julio
2,ALICANTE,25.777686,0.000820,12.192473,19.961983,31.598347,2018,julio
3,ALMERIA,24.681379,0.000000,11.574194,19.720000,29.640000,2018,julio
4,ARABA/ALAVA,20.703226,2.185000,8.122581,14.385484,27.027419,2018,julio


## AGOSTO18

In [539]:
url_ag18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-08-01T00%3A00%3A00UTC/fechafin/2018-08-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

agosto18 = solicitud_get(url_ag18, api_file_path)
print(agosto18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/d4add51a', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [540]:
url_datos_ag18= 'https://opendata.aemet.es/opendata/sh/d4add51a'
agosto_18= procesar_datos_mes(url_datos_ag18, columnas_interes)

In [541]:
agosto_18 = procesar_y_setear_mes(agosto_18, año=2018, mes='agosto')
agosto_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,20.326471,0.485909,9.269853,15.633193,25.016807,2018,agosto
1,ALBACETE,25.845161,0.926016,10.615517,18.541935,33.153226,2018,agosto
2,ALICANTE,26.163415,1.804959,10.328571,20.834959,31.489431,2018,agosto
3,ALMERIA,26.763934,0.000000,11.429032,21.574590,31.956557,2018,agosto
4,ARABA/ALAVA,20.100000,0.055357,7.950000,12.666129,27.538710,2018,agosto


## SEPTIEMBRE18

In [542]:
url_sep18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-09-01T00%3A00%3A00UTC/fechafin/2018-09-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

septiembre18 = solicitud_get(url_sep18, api_file_path)
print(septiembre18)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/ab92cd66', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [543]:
url_datos_sep18= 'https://opendata.aemet.es/opendata/sh/ab92cd66'
septiembre_18= procesar_datos_mes(url_datos_sep18, columnas_interes)

In [544]:
septiembre_18 = procesar_y_setear_mes(septiembre_18, año=2018, mes='septiembre')
septiembre_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,19.470306,0.188043,8.218045,14.895197,24.047162,2018,septiembre
1,ALBACETE,22.040336,1.602564,8.309615,15.802521,28.285714,2018,septiembre
2,ALICANTE,23.689167,1.967797,8.681111,18.853333,28.525000,2018,septiembre
3,ALMERIA,24.269159,0.925926,8.240000,19.415888,29.135514,2018,septiembre
4,ARABA/ALAVA,18.963333,0.245614,8.038333,11.033333,26.903333,2018,septiembre


## OCTUBRE18

In [545]:
url_oct18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-10-01T00%3A00%3A00UTC/fechafin/2018-10-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

octubre18 = solicitud_get(url_oct18, api_file_path)
print(octubre18) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/c5fa3f02', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [546]:
url_datos_oct18= 'https://opendata.aemet.es/opendata/sh/c5fa3f02'
octubre_18 = procesar_datos_mes(url_datos_oct18, columnas_interes)

In [547]:
octubre_18 = procesar_y_setear_mes(octubre_18, año=2018, mes='octubre')
octubre_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,14.978138,3.752500,6.746715,10.872874,19.086235,2018,octubre
1,ALBACETE,14.942276,1.233065,6.663462,9.178049,20.707317,2018,octubre
2,ALICANTE,17.866129,2.457627,7.353763,12.545161,23.183871,2018,octubre
3,ALMERIA,19.172662,0.751111,6.832258,14.812950,23.544604,2018,octubre
4,ARABA/ALAVA,12.553226,3.572727,5.172581,6.743548,18.374194,2018,octubre


## NOVIEMBRE18

In [548]:
url_nov18 = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-11-01T00%3A00%3A00UTC/fechafin/2018-11-30T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

noviembre18 = solicitud_get(url_nov18, api_file_path)
print(noviembre18) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/f6b1117e', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [549]:
url_datos_nov18= 'https://opendata.aemet.es/opendata/sh/f6b1117e'
noviembre_18= procesar_datos_mes(url_datos_nov18, columnas_interes)

In [550]:
noviembre_18 = procesar_y_setear_mes(noviembre_18, año=2018, mes='noviembre')
noviembre_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.880426,9.925000,3.191667,8.994043,14.765957,2018,noviembre
1,ALBACETE,10.178641,1.840404,4.301923,5.811650,14.552427,2018,noviembre
2,ALICANTE,13.529661,2.042857,5.422222,8.515254,18.547458,2018,noviembre
3,ALMERIA,14.713287,1.024060,6.233333,10.380420,19.034965,2018,noviembre
4,ARABA/ALAVA,9.341667,2.651786,3.848333,4.755000,13.920000,2018,noviembre


## DICIEMBRE18

In [551]:
url_dic18= 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2018-12-01T00%3A00%3A00UTC/fechafin/2018-12-31T00%3A00%3A00UTC/todasestaciones'
api_file_path = '../api.txt'

diciembre18 = solicitud_get(url_dic18, api_file_path)
print(diciembre18) 

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/77d063ff', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}


In [552]:
url_datos_dic18= 'https://opendata.aemet.es/opendata/sh/77d063ff'
diciembre_18= procesar_datos_mes(url_datos_dic18, columnas_interes)

In [553]:
diciembre_18 = procesar_y_setear_mes(diciembre_18, año=2018, mes='diciembre')
diciembre_18.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,11.794672,4.259227,3.036029,9.082377,14.511475,2018,diciembre
1,ALBACETE,8.378495,0.277419,5.596154,2.412903,14.336559,2018,diciembre
2,ALICANTE,12.069919,0.472727,7.054839,5.852033,18.294309,2018,diciembre
3,ALMERIA,13.578710,0.137419,7.135484,8.172258,18.988387,2018,diciembre
4,ARABA/ALAVA,6.827419,0.796610,3.585484,1.477419,12.175806,2018,diciembre


# CONCATENAR DF

In [586]:
dataframes = [
    enero_18, febrero_18, marzo_18, abril_18, mayo_18, junio_18, julio_18, agosto_18, septiembre_18, noviembre_18, diciembre_18,
    enero_19, febrero_19, marzo_19, abril_19, mayo_19, junio_19, julio_19, agosto_19, septiembre_19, noviembre_19, diciembre_19,
    enero_20, febrero_20, marzo_20, abril_20, mayo_20, junio_20, julio_20, agosto_20, septiembre_20, noviembre_20, diciembre_20,
    enero_21, febrero_21, marzo_21, abril_21, mayo_21, junio_21, julio_21, agosto_21, septiembre_21, noviembre_21, diciembre_21,
    enero_22, febrero_22, marzo_22, abril_22, mayo_22, junio_22, julio_22, agosto_22, septiembre_22, noviembre_22, diciembre_22
]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes)

# Resetear el índice
df_concatenado.reset_index(drop=True, inplace=True)

# Mostrar el DataFrame resultante
df_concatenado.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes
0,A CORUÑA,10.068163,3.769456,2.528467,7.360408,12.773469,2018,enero
1,ALBACETE,7.240323,1.161789,5.959649,2.165323,12.312903,2018,enero
2,ALICANTE,11.533871,2.181197,7.197849,6.019355,17.037903,2018,enero
3,ALMERIA,12.171895,0.392763,6.748387,7.255556,17.081699,2018,enero
4,ARABA/ALAVA,6.133871,4.018033,2.643548,2.108065,10.162903,2018,enero


In [587]:
locale.setlocale(locale.LC_TIME, 'es_ES')
# Combina las columnas 'año' y 'mes' en una nueva columna 'fecha_str'
df_concatenado['fecha_str'] = df_concatenado['año'].astype(str) + '-' + df_concatenado['mes']

# Convierte la nueva columna 'fecha_str' a tipo datetime
df_concatenado['fecha'] = pd.to_datetime(df_concatenado['fecha_str'], format='%Y-%B', errors='coerce')

# Muestra el DataFrame resultante
df_concatenado.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes,fecha_str,fecha
0,A CORUÑA,10.068163,3.769456,2.528467,7.360408,12.773469,2018,enero,2018-enero,2018-01-01
1,ALBACETE,7.240323,1.161789,5.959649,2.165323,12.312903,2018,enero,2018-enero,2018-01-01
2,ALICANTE,11.533871,2.181197,7.197849,6.019355,17.037903,2018,enero,2018-enero,2018-01-01
3,ALMERIA,12.171895,0.392763,6.748387,7.255556,17.081699,2018,enero,2018-enero,2018-01-01
4,ARABA/ALAVA,6.133871,4.018033,2.643548,2.108065,10.162903,2018,enero,2018-enero,2018-01-01


In [590]:
columnas_float = ['media_tmed', 'media_prec', 'media_sol', 'media_tmin', 'media_tmax']

# Redondear a dos decimales las columnas especificadas
df_concatenado[columnas_float] = df_concatenado[columnas_float].round(2)
df_concatenado.head()

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,año,mes,fecha_str,fecha
0,A CORUÑA,10.07,3.77,2.53,7.36,12.77,2018,enero,2018-enero,2018-01-01
1,ALBACETE,7.24,1.16,5.96,2.17,12.31,2018,enero,2018-enero,2018-01-01
2,ALICANTE,11.53,2.18,7.20,6.02,17.04,2018,enero,2018-enero,2018-01-01
3,ALMERIA,12.17,0.39,6.75,7.26,17.08,2018,enero,2018-enero,2018-01-01
4,ARABA/ALAVA,6.13,4.02,2.64,2.11,10.16,2018,enero,2018-enero,2018-01-01


In [591]:
# Guardar el DataFrame combinado en un nuevo archivo csv
df_concatenado.to_csv('clima.csv')